In [ ]:
#TESTING WEB SCRAPING

#For single recipe, taking nutrient info

In [2]:
import requests
from bs4 import BeautifulSoup
import time

import pandas as pd

In [2]:
url = 'https://www.allrecipes.com/recipe/18057/sweet-potato-casserole-ii/'
url1 = url + 'fullrecipenutrition'
r = requests.get(url1)
print('Response body: {0}'.format(r.text))

Response body: 
<div class="recipe-nutrition">
    <h1 class="nutrition_h1">Nutritional Information</h1>
    <h2>Sweet Potato Casserole II</h2>
    <div class="nutrition-top light-underline">

        Servings Per Recipe: 16<br />
        <span class="semi-bold">Amount Per Serving</span><br />
            <span class="semi-bold">Calories:</span> 309<br />
    </div>
    <div class="nutrition-body-label">% Daily Value *</div>
    <div class="nutrition-body">
        <div class="nutrition-row">
            <span class="nutrient-name">Total Fat: <span aria-label="15.3 grams"></span><span class="nutrient-value" aria-hidden="true">15.3g</span></span>
                <span class="daily-value">24 %</span>
        </div>
                <div class="nutrition-row">
            <span class="nutrient-name indent">Saturated Fat: <span aria-label="7.0 grams"></span><span class="nutrient-value" aria-hidden="true">7.0g</span></span>
        </div>
                <div class="nutrition-row">
         

In [3]:
page_body = r.text
soup = BeautifulSoup(page_body, 'html.parser')

In [4]:
#this is the div containing all the nutrition information
nutrition_info = soup.find_all('div', class_='nutrition-row')

In [5]:
#nutrition_info

In [6]:
nutrient_list = []
recipe = soup.find('h2').text
for n in nutrition_info:
    name = n.find(class_ = 'nutrient-name').text
    amount = n.find(class_ = 'nutrient-value').text
    name = name[:name.index(':')]
    nutrient_list.append({'nutrient': name, 'amount': amount, 'recipe': recipe, 'URL': url})
nutrients = pd.DataFrame(nutrient_list)

nutrients.head()

,URL,amount,nutrient,recipe
0,https://www.allrecipes.com/recipe/18057/sweet-...,15.3g,Total Fat,Sweet Potato Casserole II
1,https://www.allrecipes.com/recipe/18057/sweet-...,7.0g,Saturated Fat,Sweet Potato Casserole II
2,https://www.allrecipes.com/recipe/18057/sweet-...,49mg,Cholesterol,Sweet Potato Casserole II
3,https://www.allrecipes.com/recipe/18057/sweet-...,103mg,Sodium,Sweet Potato Casserole II
4,https://www.allrecipes.com/recipe/18057/sweet-...,337mg,Potassium,Sweet Potato Casserole II


In [7]:
# & ingredients

In [8]:
s = requests.get(url)

In [9]:
page_body2 = s.text
soup2 = BeautifulSoup(page_body2, 'html.parser')

In [10]:
ingredient_info = soup2.find_all(class_='recipe-container-outer')
#ingredient_info

In [11]:
ingredient_list = []
for i in ingredient_info:
    ingredients = i.find_all('span', {'itemprop':'recipeIngredient'}, class_ = 'recipe-ingred_txt added')
    for x in ingredients:
        ingredient_list.append({'ingredient': x.text, 'URL':url})

ingredients = pd.DataFrame(ingredient_list)

ingredients.head()

,URL,ingredient
0,https://www.allrecipes.com/recipe/18057/sweet-...,4 1/2 cups cooked and mashed sweet potatoes
1,https://www.allrecipes.com/recipe/18057/sweet-...,"1/2 cup butter, melted"
2,https://www.allrecipes.com/recipe/18057/sweet-...,1/3 cup milk
3,https://www.allrecipes.com/recipe/18057/sweet-...,1 cup white sugar
4,https://www.allrecipes.com/recipe/18057/sweet-...,1/2 teaspoon vanilla extract


In [3]:
# FUNCTION FOR WEB SCRAPING

import time
def get_recipe_data(url, nutrient_df, ingredient_df):
    """
    Scrape allrecipes.com for information on the nutrient content and ingredients of each recipe
    Based on the url, ingredients and nutrient content are extracted separately from the website.
    
    IN: url: URL of the specific recipe as a string
    OUT: nutrient and ingredient dataframes with the information for the given recipe
    """
    
    #nutrition information
    url_nutrition = url + 'fullrecipenutrition'
    time.sleep(1)
    r1 = requests.get(url_nutrition)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    
    #this is the div containing all the nutrition information
    nutrition_info = soup1.find_all('div', class_='nutrition-row')
    
    nutrient_list = []
    
    recipe = soup1.find('h2')
    if recipe is not None:
        recipe = recipe.text
        for n in nutrition_info:
            name = n.find(class_ = 'nutrient-name').text
            amount = n.find(class_ = 'nutrient-value').text
            name = name[:name.index(':')]
            nutrient_list.append({'nutrient': name, 'amount': amount, 'recipe': recipe, 'URL': url})

    nutrients = pd.DataFrame(nutrient_list)
    nutrient_df = nutrient_df.append(nutrients, sort=True)
    
    #ingredients
    r2 = requests.get(url)
    soup2 = BeautifulSoup(r2.text, 'html.parser')
    ingredient_info = soup2.find_all(class_='recipe-container-outer')
    
    ingredient_list = []
    for i in ingredient_info:
        ingredient = i.find_all('span', {'itemprop':'recipeIngredient'}, class_ = 'recipe-ingred_txt added')
        for x in ingredient:
            ingredient_list.append({'ingredient': x.text, 'URL':url})

    ingredients = pd.DataFrame(ingredient_list)
    ingredient_df = ingredient_df.append(ingredients, sort=True)
    
    return nutrient_df, ingredient_df

In [23]:
#test the function
url = 'https://www.allrecipes.com/recipe/18057/sweet-potato-casserole-ii/'
nutrient_columns = ['URL', 'amount', 'nutrient', 'recipe']
ingredient_columns = ['URL', 'ingredient']
nutrient_df = pd.DataFrame(columns=nutrient_columns)
ingredient_df = pd.DataFrame(columns=ingredient_columns)

nutrient_df, ingredient_df = get_recipe_data(url, nutrient_df, ingredient_df)

In [24]:
#url2 = 'https://www.allrecipes.com/recipe/7589/allspice-cream-cheese-frosting/'
#nutrient_df, ingredient_df = get_recipe_data(url2, nutrient_df, ingredient_df)

In [25]:
#nutrient_df

In [9]:
dinner_url_df = pd.read_csv('data/recipeLists/dinner_url.csv')
dinner_url_df.columns = ["originGridUrl", "recipeName", "recipeURL"]
#pd.options.display.max_colwidth = 200
#display(dinner_url_df['recipeURL'].head())

In [10]:
#dinner_url_df['recipeURL'] = dinner_url_df['recipeURL'].split("?")[0]
dinner_url_df['recipeURL'] = dinner_url_df['recipeURL'].astype(str).apply(lambda x: x.split("?")[0])
#dinner_url_df['recipeURLPrime'] = str(dinner_url_df['recipeURL']).split("?")[0]
pd.set_option('display.max_rows', None,'display.max_columns', None)
#display(dinner_url_df.head())

In [11]:
dinnerURLList = dinner_url_df.recipeURL.tolist()

In [12]:
dinnerURLList[:5]

['https://www.allrecipes.com/recipe/222607/smothered-chicken-breasts/',
 'https://www.allrecipes.com/recipe/15679/asian-beef-with-snow-peas/',
 'https://www.allrecipes.com/recipe/23847/pasta-pomodoro/',
 'https://www.allrecipes.com/recipe/50435/fry-bread-tacos-ii/',
 'https://www.allrecipes.com/recipe/140829/pork-marsala/']

In [13]:
nutrient_columns = ['URL', 'amount', 'nutrient', 'recipe']
ingredient_columns = ['URL', 'ingredient']
nutrient_df = pd.DataFrame(columns=nutrient_columns)
ingredient_df = pd.DataFrame(columns=ingredient_columns)

for url in dinnerURLList:
    nutrient_df, ingredient_df = get_recipe_data(url, nutrient_df, ingredient_df)
    time.sleep(1)
    

MissingSchema: Invalid URL 'nanfullrecipenutrition': No schema supplied. Perhaps you meant http://nanfullrecipenutrition?

In [14]:
nutrient_df.to_csv("dinner_nutrient.csv")
ingredient_df.to_csv("dinner_ingredient.csv")

In [8]:
nutrient_df.shape

(14634, 4)

In [39]:
dinnerURLList

['https://www.allrecipes.com/recipe/222607/smothered-chicken-breasts/',
 'https://www.allrecipes.com/recipe/15679/asian-beef-with-snow-peas/',
 'https://www.allrecipes.com/recipe/23847/pasta-pomodoro/',
 'https://www.allrecipes.com/recipe/50435/fry-bread-tacos-ii/',
 'https://www.allrecipes.com/recipe/140829/pork-marsala/',
 'https://www.allrecipes.com/recipe/217981/grilled-chicken-with-rosemary-and-bacon/',
 'https://www.allrecipes.com/recipe/87624/beefy-baked-ravioli/',
 'https://www.allrecipes.com/recipe/87648/fresh-tomato-shrimp-pasta/',
 'https://www.allrecipes.com/recipe/80388/lemon-rosemary-salmon/',
 'https://www.allrecipes.com/recipe/23600/worlds-best-lasagna/',
 'https://www.allrecipes.com/recipe/50644/broiled-tilapia-parmesan/',
 'https://www.allrecipes.com/recipe/16066/awesome-slow-cooker-pot-roast/',
 'https://www.allrecipes.com/recipe/25203/brown-sugar-meatloaf/',
 'https://www.allrecipes.com/recipe/11758/baked-ziti-i/',
 'https://www.allrecipes.com/recipe/9023/baked-teri